In [43]:
from openai import OpenAI
import pandas as pd
from sqlalchemy import create_engine, text

In [44]:
openai = OpenAI()

In [45]:
df = pd.read_csv('../data/sales.csv')

In [46]:
df.head(3)

,OrderDate,Category,City,Country,CustomerName,Discount,OrderID,PostalCode,ProductName,Profit,...,DaystoShipActual,SalesForecast,ShipStatus,DaystoShipScheduled,OrderProfitable,SalesperCustomer,ProfitRatio,SalesaboveTarget,latitude,longitude
0,2011-01-04T00:00:00.000Z,Office Supplies,Houston,United States,Darren Powers,0.2,CA-2011-103800,77095,"Message Book, Wirebound, Four 5 1/2 X 4 Forms/...",6,...,4,22,Shipped Early,6,NaN,16.45,33.8,NaN,29.8941,-95.6481
1,2011-01-05T00:00:00.000Z,Office Supplies,Naperville,United States,Phillina Ober,0.2,CA-2011-112326,60540,Avery 508,4,...,4,15,Shipped Early,6,NaN,11.78,36.3,NaN,41.7662,-88.1410
2,2011-01-05T00:00:00.000Z,Office Supplies,Naperville,United States,Phillina Ober,0.8,CA-2011-112326,60540,GBC Standard Plastic Binding Systems Combs,-5,...,4,5,Shipped Early,6,NaN,3.54,-155.0,NaN,41.7662,-88.1410


In [47]:
db = create_engine('sqlite:///:memory:')

In [48]:
data = df.to_sql(name='sales', con=db)

In [49]:
with db.connect() as conn:
    result = conn.execute(text('SELECT SUM(Sales) FROM SALES'))

result.all()

[(2297354,)]

In [50]:
def create_table_definition(df):
    prompt = """### sqlite SQL table, with it properties:
    #
    #Sales({})
    #
    """.format(','.join([str(col) for col in df.columns]))

    return prompt

In [85]:
def combine_prompts(df, prompt_query):
    definition = create_table_definition(df)
    query_string = f"### A query to answer: {prompt_query}\nSELECT"
    return definition + query_string

In [86]:
def prompt_input():
    return input("Enter the info you want?")

In [90]:
nlp_text = prompt_input()
print(combine_prompts(df, nlp_text))

Enter the info you want? give me the sum of sales per quarter


### sqlite SQL table, with it properties:
    #
    #Sales(OrderDate,Category,City,Country,CustomerName,Discount,OrderID,PostalCode,ProductName,Profit,Quantity,Region,Sales,Segment,ShipDate,ShipMode,State,Sub_Category,DaystoShipActual,SalesForecast,ShipStatus,DaystoShipScheduled,OrderProfitable,SalesperCustomer,ProfitRatio,SalesaboveTarget,latitude,longitude)
    #
    ### A query to answer: give me the sum of sales per quarter
SELECT


In [103]:
response = openai.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{'role': 'user', 'content': combine_prompts(df, nlp_text)}],
    temperature=0.4,
    stop=["#", ";'"]
)

In [104]:
nlp_to_query = f'SELECT {response.choices[0].message.content.strip()}'
print(nlp_to_query)

SELECT SUM(Sales) as total_sales, 
CASE 
    WHEN strftime('%m', OrderDate) IN ('01','02','03') THEN 'Q1'
    WHEN strftime('%m', OrderDate) IN ('04','05','06') THEN 'Q2'
    WHEN strftime('%m', OrderDate) IN ('07','08','09') THEN 'Q3'
    WHEN strftime('%m', OrderDate) IN ('10','11','12') THEN 'Q4'
END as quarter
FROM Sales
GROUP BY quarter;


In [105]:
with db.connect() as conn:
    result = conn.execute(text(nlp_to_query))

result.all()

[(354817, 'Q1'), (445090, 'Q2'), (618996, 'Q3'), (878451, 'Q4')]